# Cross Encoder 重排器

本 Notebook 展示了如何在检索器中实现重排器，使用您自己的交叉编码器，这些交叉编码器来自 [Hugging Face 交叉编码器模型](https://huggingface.co/cross-encoder) 或实现了交叉编码器功能的 Hugging Face 模型 ([示例：BAAI/bge-reranker-base](https://huggingface.co/BAAI/bge-reranker-base))。`SagemakerEndpointCrossEncoder` 使您能够使用加载到 Sagemaker 上的这些 HuggingFace 模型。

这建立在 [ContextualCompressionRetriever](/docs/how_to/contextual_compression) 的思想之上。本文档的整体结构来自于 [Cohere Reranker 文档](/docs/integrations/retrievers/cohere-reranker)。

有关为什么交叉编码器可以与嵌入结合使用作为重排机制以实现更好检索的更多信息，请参阅 [Hugging Face 交叉编码器文档](https://www.sbert.net/examples/applications/cross-encoder/README.html)。

In [ ]:
#!pip install faiss sentence_transformers

# OR  (depending on Python version)

#!pip install faiss-cpu sentence_transformers

In [10]:
# Helper function for printing docs


def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

## 设置基础向量存储检索器
让我们从初始化一个简单的向量存储检索器开始，并将 2023 年国情咨文（分块）存储其中。我们可以将检索器设置为检索较多的文档（20 个）。

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

documents = TextLoader("../../how_to/state_of_the_union.txt").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)
embeddingsModel = HuggingFaceEmbeddings(
    model_name="sentence-transformers/msmarco-distilbert-dot-v5"
)
retriever = FAISS.from_documents(texts, embeddingsModel).as_retriever(
    search_kwargs={"k": 20}
)

query = "What is the plan for the economy?"
docs = retriever.invoke(query)
pretty_print_docs(docs)

## 使用 CrossEncoderReranker 进行重排
现在，我们将基础检索器包装在 `ContextualCompressionRetriever` 中。`CrossEncoderReranker` 使用 `HuggingFaceCrossEncoder` 来重排返回的结果。

In [31]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
compressor = CrossEncoderReranker(model=model, top_n=3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke("What is the plan for the economy?")
pretty_print_docs(compressed_docs)

Document 1:

More infrastructure and innovation in America. 

More goods moving faster and cheaper in America. 

More jobs where you can earn a good living in America. 

And instead of relying on foreign supply chains, let’s make it in America. 

Economists call it “increasing the productive capacity of our economy.” 

I call it building a better America. 

My plan to fight inflation will lower your costs and lower the deficit.
----------------------------------------------------------------------------------------------------
Document 2:

Second – cut energy costs for families an average of $500 a year by combatting climate change.  

Let’s provide investments and tax credits to weatherize your homes and businesses to be energy efficient and you get a tax credit; double America’s clean energy production in solar, wind, and so much more;  lower the price of electric vehicles, saving you another $80 a month because you’ll never have to pay at the gas pump again.
------------------------

## 将 Hugging Face 模型上传到 SageMaker endpoint

以下是一个示例 `inference.py` 文件，用于创建与 `SagemakerEndpointCrossEncoder` 配合使用的 endpoint。有关更详细的分步指南，请参阅[本文](https://huggingface.co/blog/kchoe/deploy-any-huggingface-model-to-sagemaker)。

它会动态下载 Hugging Face 模型，因此您无需将 `pytorch_model.bin` 等模型构件保存在 `model.tar.gz` 中。

In [ ]:
import json
import logging
from typing import List

import torch
from sagemaker_inference import encoder
from transformers import AutoModelForSequenceClassification, AutoTokenizer

PAIRS = "pairs"
SCORES = "scores"


class CrossEncoder:
    def __init__(self) -> None:
        self.device = (
            torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        )
        logging.info(f"Using device: {self.device}")
        model_name = "BAAI/bge-reranker-base"
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.model = self.model.to(self.device)

    def __call__(self, pairs: List[List[str]]) -> List[float]:
        with torch.inference_mode():
            inputs = self.tokenizer(
                pairs,
                padding=True,
                truncation=True,
                return_tensors="pt",
                max_length=512,
            )
            inputs = inputs.to(self.device)
            scores = (
                self.model(**inputs, return_dict=True)
                .logits.view(
                    -1,
                )
                .float()
            )

        return scores.detach().cpu().tolist()


def model_fn(model_dir: str) -> CrossEncoder:
    try:
        return CrossEncoder()
    except Exception:
        logging.exception(f"Failed to load model from: {model_dir}")
        raise


def transform_fn(
    cross_encoder: CrossEncoder, input_data: bytes, content_type: str, accept: str
) -> bytes:
    payload = json.loads(input_data)
    model_output = cross_encoder(**payload)
    output = {SCORES: model_output}
    return encoder.encode(output, accept)